In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dcor as dc
from scipy.cluster.hierarchy import linkage, cut_tree, dendrogram
from scipy.spatial.distance import squareform
from scipy.stats import norm
from math import sqrt,  tanh, ceil, log, cos, pi, sin
import seaborn as sns
import time
from numpy.linalg import inv
from numpy import linalg as LA
import random
from IPython.display import display, Math, Latex
from scipy.io import loadmat
from scipy.linalg import pinv2, sqrtm
from scipy.optimize import minimize
from scipy.linalg import orth

In [ ]:
# import functions from necessary files 
import os
original_dir = os.getcwd()
os.chdir('..\\src\\robustOptimPack\\Utils')
from helper_funcs import *
os.chdir('..\\src\\robustOptimPack\\MVPortfolio')
from MVPortfolio_funcs import *
os.chdir('..\\src\\robustOptimPack\\HCPortfolio')
from HCPortfolio_funcs import *
os.chdir('..\\src\\robustOptimPack\\PortfolioBacktest')
from Backtest_funcs import *
os.chdir('..\\src\\robustOptimPack\\wrapping')
from wrapping_funcs import *
os.chdir(original_dir)

In [ ]:
n_reps = 100
n_obs_vec = [1000]
dates_vec = pd.date_range('2018-01-01', periods=1000, freq='D')
res_mat_noout= np.ones((len(n_obs_vec),n_reps, 4 ))
delta_t = 1/360
mean_vec = np.array([-0.45, -0.25, -0.2, -0.15, -0.1, -0.05, 0.05, 0.08, 0.1, 0.15, 0.2, 0.21, 0.25, 0.3, 0.45])*delta_t
sd_vec = np.array([0.5, 0.2, 0.3, 0.25, 0.3, 0.2, 0.25, 0.23, 0.2, 0.4, 0.2, 0.25, 0.3, 0.15, 0.5]) *np.sqrt(delta_t)
sd_white_noise = sqrt(0.01)
nu_vec = np.linspace(start=5, stop=75, num=15)
_lambda_vec = np.round(np.linspace(start=-1, stop=1, num=15),2)
n_clusts = 8
n_assets = 20
link_method = 'average'
out_fraction = 0.2
assets_in_groups = np.repeat([5,4,3,2,1], [1,1,2,1,3])
out_dist = 64
test_window = 30
train_window = 180

#### Basic simulation without outliers 

In [ ]:
 for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     
     for cur_rep in range(n_reps):
        
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         data_assets = np.zeros((n_obs, 1))
         
         for group_ind in assets_in_groups: 
             #print('group_ind is ', group_ind)
             #initialise data matrix for that group: 
             data_group_ind = np.zeros((n_obs, group_ind))
             #generate skewed t variate for that group  
             # sample the kurtosis parameter 
             nu_l = np.random.choice(nu_vec, size = 1)
             _lambda_l  =  np.random.choice(_lambda_vec, size = 1)
             #print('nu: ', nu_l, ':lambda ', _lambda_l)
             y_l_t = generate_skt(n_obs, nu_l, _lambda_l).flatten() 
             
             for asset_ind in range(group_ind): 
                 
                 #sample mean and volatility for current asset in current group 
                 asset_i_mean = np.random.choice(mean_vec, size = 1)
                 asset_i_vol = np.random.choice(sd_vec, size = 1)
                 
                 data_group_ind[:,asset_ind] = asset_i_mean + np.multiply(y_l_t, asset_i_vol)
                 #print('data_group shape ', data_group_ind.shape)
             data_assets = np.column_stack((data_assets, data_group_ind))
             #print('data_asset shape ', data_assets.shape)
         
         dat_sim = pd.DataFrame(data_assets[:,1:])
         dat_sim.set_index(dates_vec, inplace=True)
         volumes_df = pd.DataFrame(np.ones((dat_sim.shape[0], dat_sim.shape[1])))# we want to use all assets
         #initialise portfolio objects 
         portfolio_equal_crypto = EqualWeightPortfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='EW', init_amount_invested=1)


         portfolio_cov_crypto = MVPortfolio(asset_names=list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='MV',init_amount_invested=1,long_only=True)

         portfolio_HRP_crypto_cor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_cor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'sample', linkage_method = "average")

         portfolio_HRP_crypto_dcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_dcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'dcor', linkage_method = "average")

         portfolio_HRP_crypto_wcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average")

         portfolio_HRP_crypto_wdcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data= dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wdcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average")

         portfolio_HERC_crypto_cor_average_sil = HERC_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data= dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_cor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'sample', linkage_method = "average",num_clusters_method = 'silhouette' )

         portfolio_HERC_crypto_dcor_average_sil = HERC_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_dcor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'dcor', linkage_method = "average", num_clusters_method = 'silhouette')

         portfolio_HERC_crypto_wcor_average_sil = HERC_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wcor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average",num_clusters_method = 'silhouette')

         portfolio_HERC_crypto_wdcor_average_sil = HERC_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wdcor_sil', init_amount_invested=1, 
                                                                cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average",num_clusters_method = 'silhouette')

         portfolio_cluster_sharpe_sil = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe', init_amount_invested=1, 
                                                    cor_method = 'sample', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR', init_amount_invested=1, 
                                                    cor_method = 'sample', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )
        
         portfolio_cluster_sharpe_sil_dcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_dcor', init_amount_invested=1, 
                                                    cor_method = 'dcor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_dcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_dcor', init_amount_invested=1, 
                                                    cor_method = 'dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

         portfolio_cluster_sharpe_sil_wcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wcor', init_amount_invested=1, 
                                                    cor_method = 'robust_cor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_wcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wcor', init_amount_invested=1, 
                                                    cor_method = 'robust_cor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

         portfolio_cluster_sharpe_sil_wdcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wdcor', init_amount_invested=1, 
                                                    cor_method = 'robust_dcor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_wdcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wdcor', init_amount_invested=1, 
                                                    cor_method = 'robust_dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

         list_of_portfolios = [portfolio_equal_crypto, portfolio_cov_crypto, 
                            portfolio_HRP_crypto_cor_average,portfolio_HRP_crypto_dcor_average,portfolio_HRP_crypto_wcor_average,portfolio_HRP_crypto_wdcor_average,
                            portfolio_HERC_crypto_cor_average_sil,portfolio_HERC_crypto_dcor_average_sil,portfolio_HERC_crypto_wcor_average_sil,portfolio_HERC_crypto_wdcor_average_sil,
                            portfolio_cluster_sharpe_sil,
                            portfolio_cluster_CVaR_sil, 
                            portfolio_cluster_sharpe_sil_dcor,
                            portfolio_cluster_CVaR_sil_dcor, 
                            portfolio_cluster_sharpe_sil_wcor,
                            portfolio_cluster_CVaR_sil_wcor, 
                            portfolio_cluster_sharpe_sil_wdcor, 
                            portfolio_cluster_CVaR_sil_wdcor
                            ]

         portfolios_backtest_crypto = backtest_crypto_portfolios(list_of_portfolios,returns_data=dat_sim,
                                                                log_returns_data = dat_sim,
                                                                is_price_data=False, volumes = volumes_df, dates = dates_vec, K=n_assets, 
                                                                train_window_size = train_window, transaction_cost = 0,
                                                                test_window_size = test_window,rebalance_window_size=test_window)
         portfolios_backtest_crypto.run_backtest()
         
         if (cur_rep == 0): 
             results_df = portfolios_backtest_crypto.out_of_sample_results()
         else: 
             results_df = pd.concat([results_df,portfolios_backtest_crypto.out_of_sample_results()])
             
 

         
         
         

In [ ]:
results_df['index1'] = results_df.index
stat_desc = round(results_df.groupby('index1').median(),2)
print(stat_desc.to_latex())

In [ ]:
sns.pairplot(pd.DataFrame(dat_sim))

#### Simulation with cellwise outliers in all variables 


In [ ]:
out_dim = n_assets

In [ ]:
for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     
     for cur_rep in range(n_reps):
        
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         data_assets = np.zeros((n_obs, 1))
         
         # generate index of outlying columns 
         out_cols = np.sort( np.random.randint(n_assets, size = out_dim))
         out_col_counter = 0
         
         
         for group_ind in assets_in_groups: 
             #print('group_ind is ', group_ind)
             #initialise data matrix for that group: 
             data_group_ind = np.zeros((n_obs, group_ind))
             nu_l = np.random.choice(nu_vec, size = 1)
             _lambda_l  =  np.random.choice(_lambda_vec, size = 1)
             #print('nu: ', nu_l, ':lambda ', _lambda_l)
             y_l_t = generate_skt(n_obs, nu_l, _lambda_l).flatten()
             
             for asset_ind in range(group_ind): 
                 
                 #sample mean and volatility for current asset in current group 
                 asset_i_mean = np.random.choice(mean_vec, size = 1)
                 asset_i_vol = np.random.choice(sd_vec, size = 1)
                 
                 data_group_ind[:,asset_ind] = asset_i_mean + np.multiply(y_l_t, asset_i_vol)
                 #print('data_group shape ', data_group_ind.shape)
                 
                 # generate outliers if the current column should contain outliers 
                 if out_col_counter in out_cols: 
                     #print('out col ', out_col_counter, ' max ', np.max(data_group_ind[:,asset_ind]))
                     outliers_ind = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
                     data_group_ind[outliers_ind,asset_ind] =  random.sample([-1,1], 1)[0] *(np.max(data_group_ind[:,asset_ind]) *out_dist)/sqrt(out_dim)
                     #print('out col ', out_col_counter, ' max ', np.max(data_group_ind[:,asset_ind]))
                 out_col_counter +=1 
             data_assets = np.column_stack((data_assets, data_group_ind))
             #print('data_asset shape ', data_assets.shape)
         
         dat_sim = pd.DataFrame(data_assets[:,1:])
         dat_sim.set_index(dates_vec, inplace=True)
         volumes_df = pd.DataFrame(np.ones((dat_sim.shape[0], dat_sim.shape[1])))# we want to use all assets
         #initialise portfolio objects 
         portfolio_equal_crypto = EqualWeightPortfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='EW', init_amount_invested=1)

         portfolio_cov_crypto = MVPortfolio(asset_names=list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='MV',init_amount_invested=1,long_only=True)

         portfolio_HRP_crypto_cor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_cor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'sample', linkage_method = "average")

         portfolio_HRP_crypto_dcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_dcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'dcor', linkage_method = "average")

         portfolio_HRP_crypto_wcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average")

         portfolio_HRP_crypto_wdcor_average = HRP_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data= dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HRP_wdcor', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average")

         portfolio_HERC_crypto_cor_average_sil = HERC_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data= dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_cor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'sample', linkage_method = "average",num_clusters_method = 'silhouette' )

         portfolio_HERC_crypto_dcor_average_sil = HERC_portfolio(asset_names= list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_dcor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'dcor', linkage_method = "average", num_clusters_method = 'silhouette')

         portfolio_HERC_crypto_wcor_average_sil = HERC_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets], data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wcor_sil', init_amount_invested=1, 
                                                    cov_method = 'sample', cor_method = 'robust_cor', linkage_method = "average",num_clusters_method = 'silhouette')

         portfolio_HERC_crypto_wdcor_average_sil = HERC_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='HERC_wdcor_sil', init_amount_invested=1, 
                                                                cov_method = 'sample', cor_method = 'robust_dcor', linkage_method = "average",num_clusters_method = 'silhouette')

         portfolio_cluster_sharpe_sil = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe', init_amount_invested=1, 
                                                    cor_method = 'sample', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR', init_amount_invested=1, 
                                                    cor_method = 'sample', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )
        
         portfolio_cluster_sharpe_sil_dcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_dcor', init_amount_invested=1, 
                                                    cor_method = 'dcor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_dcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_dcor', init_amount_invested=1, 
                                                    cor_method = 'dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

         portfolio_cluster_sharpe_sil_wcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wcor', init_amount_invested=1, 
                                                    cor_method = 'robust_cor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_wcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets] , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wcor', init_amount_invested=1, 
                                                    cor_method = 'robust_cor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )

         portfolio_cluster_sharpe_sil_wdcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_sharpe_wdcor', init_amount_invested=1, 
                                                    cor_method = 'robust_dcor', linkage_method = "complete", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "sharpe" )

         portfolio_cluster_CVaR_sil_wdcor = max_cluster_portfolio(asset_names=list(volumes_df.columns.values)[0:n_assets]  , data=dat_sim, is_price_data=False, risk_free_rate=0, period_length=1, name='cluster_CVaR_wdcor', init_amount_invested=1, 
                                                    cor_method = 'robust_dcor', linkage_method = "average", num_clusters_type = 'search', num_clusters_method = 'silhouette', risk_method = "CVaR" )


         list_of_portfolios = [portfolio_equal_crypto, portfolio_cov_crypto, 
                            portfolio_HRP_crypto_cor_average,portfolio_HRP_crypto_dcor_average,portfolio_HRP_crypto_wcor_average,portfolio_HRP_crypto_wdcor_average,
                            portfolio_HERC_crypto_cor_average_sil,portfolio_HERC_crypto_dcor_average_sil,portfolio_HERC_crypto_wcor_average_sil,portfolio_HERC_crypto_wdcor_average_sil,
                            portfolio_cluster_sharpe_sil,
                            portfolio_cluster_CVaR_sil, 
                            portfolio_cluster_sharpe_sil_dcor,
                            portfolio_cluster_CVaR_sil_dcor,  
                            portfolio_cluster_sharpe_sil_wcor,
                            portfolio_cluster_CVaR_sil_wcor, 
                            portfolio_cluster_sharpe_sil_wdcor, 
                            portfolio_cluster_CVaR_sil_wdcor
                            ]


         portfolios_backtest_crypto = backtest_crypto_portfolios(list_of_portfolios,returns_data=dat_sim,
                                                                log_returns_data = dat_sim,
                                                                is_price_data=False, volumes = volumes_df, dates = dates_vec, K=n_assets, 
                                                                train_window_size = train_window, transaction_cost = 0,
                                                                test_window_size = test_window,rebalance_window_size=test_window)
         portfolios_backtest_crypto.run_backtest()
         
         if (cur_rep == 0): 
             results_df_cell = portfolios_backtest_crypto.out_of_sample_results()
         else: 
             results_df_cell = pd.concat([results_df_cell,portfolios_backtest_crypto.out_of_sample_results()])


In [ ]:
results_df_cell['index1'] = results_df_cell.index
stat_desc_cell = round(results_df_cell.groupby('index1').median(),2)
print(stat_desc_cell.to_latex())